In [1]:
import numpy as np
import time
import sys
import random
sys.path.append('./utils')
from Reaction import *
from GenerateStrategies import *

In [2]:
money_mode = np.load('./strategies/money_mode.npy')
supply_mode = np.load('./strategies/supply_mode.npy')
forecast_mode = np.load('./strategies/forecast_mode.npy')

In [3]:
army_infos = Miragine().data

In [4]:
# 参数设置
abyss = False
turn_time = 24.93  # 每一轮的时间
waits = (6.2  * turn_time, 11.2 * turn_time, 14.3 * turn_time, 23.3 * turn_time)

surprise_area = (1100, 800, 1200, 860)
defense_area = (720, 800, 900, 860)

In [5]:
'''
Decide which army to choose based on given strategies. Quick and cheap armies are favored.
'''
def decide_move(strategy, opmove, abyss=False):
    # 对付Ninja指定用Samurai
    if opmove == 6:
        return 4
    # 对付Novice指定用Ninja
    if opmove == 7:
        return 6
    # 对付Mage指定用Heavy SwordMan
    if opmove == 13:
        return 8
    # 派出几个快速军种
    if (strategy[1] in (6, 8, 11, 14)) and (strategy[0] not in (6, 8, 9, 11, 12, 14, 15, 16)):
        return strategy[1]
    # 派出几个便宜军种
    if abyss and army_infos[strategy[1]]['income'] / army_infos[strategy[1]]['money'] > army_infos[strategy[0]]['income'] / army_infos[strategy[0]]['money']:
        return strategy[1]
    
    # 如无上述特殊条件，派出最优应对军种
    return strategy[0]

In [6]:
t0 = time.time()

op_last_move = None

my_hist = [0, 0, 0]
op_hist = [0, 0, 0]
surprise_hist = [0, 0, 0]

allow_surprise = False

stage = 0
count = 0

'''
Stage 0.
'''
while stage == 0:
    t1 = time.time()

    # 查看对方派兵
    try:
        opmove = get_opponent_move()
    except:
        continue

    # 阶段改变条件，一旦对手出High Lord并且危急，我就进入stage 1 应对。
    if opmove == 16 and t1 > 12.5 * turn_time:
        stage = 1
        break

    # 获取我方策略空间
    my_moves = money_mode[opmove-1]

    # 奇袭逻辑
    surprise = surprise_attack(allow_surprise, stage, opmove, surprise_area)

    # 深渊模式 stage 0 不奇袭
    if abyss:
        surprise = False

    # 禁止连续冲马奇袭
    if surprise_hist[-2] == 11:
        surprise = False

    # 没问题就奇袭
    if surprise != False:
        my_move = surprise
        real_move = my_move

    # 正常轮次逻辑
    else:
        # 开局刷Samurai赚钱，反正塔有防御，对面攻不过来
        if t1 - t0 < waits[0]:
            my_move = 4
            real_move = my_move

        # 然后刷Zombie/Sword Man/Ninja赚钱，反正塔有防御，对面攻不过来
        elif t1 - t0 < waits[1]:

            my_move = [3, 5, 6][count % 3]
            real_move = my_move

        # 中局逻辑，见招拆招，同时预防性派出预测军种
        else:
            # 先刷低阶军
            if t1 - t0 < waits[2]:
                strategy = np.extract(my_moves <= 9, my_moves)
                my_move = decide_move(strategy, opmove, abyss)
            # 中间轮不用Lord
            elif t1 - t0 < waits[3]:
                strategy = np.extract(my_moves <= 14, my_moves)
                my_move = decide_move(strategy, opmove, abyss)
                allow_surprise = True
            # 中举后半部分允许用Lord
            else:
                strategy = my_moves
                my_move = decide_move(strategy, opmove, abyss)
                allow_surprise = True

            # 生成预测军种
            forecast_move = forecast_mode[my_move-1]

            # 预测军种不要跑太快被人家现在这一波给吃掉了，也不要让人家的魔法师堆在一起
            if army_infos[forecast_move]['move_speed'] <= army_infos[my_move]['move_speed'] and opmove not in [7, 10, 13]:
                real_move = my_move if count % 7 else forecast_move
            else:
                real_move = my_move

    click_area(real_move)

    # 记录历史以便分辨轮次
    if opmove != op_last_move and opmove != op_hist[-1]:
        my_hist.append(my_move)
        op_hist.append(opmove)
        surprise_hist.append(surprise)

    # 更新历史数据
    op_last_move = opmove
    count += 1

    time.sleep(0.01)

'''
Stage 1.
'''
t_0 = time.time() - 0.1
turns = 0
surprise_area = (1050, 800, 1200, 860)


while stage == 1:

    # 进入残局，允许奇袭
    allow_surprise = True

    try:
        opmove = get_opponent_move()
    except:
        continue

    surprise = surprise_attack(allow_surprise, stage, opmove, surprise_area)

    # 防御必出High Lord
    if (my_hist[-1] != 16 and my_hist[-2] != 16) or defense(defense_area):
        my_move = 16
        click_area(my_move)
        time.sleep(6)
        click_area(7)

    # 确保每7轮必连出High Lord
    elif turns % 7 == 1 or turns % 7 == 2:
        my_move = 16
        click_area(my_move)
        time.sleep(6)
        click_area(7)

    # 防御过后，奇袭优先，但禁止连续冲马
    elif surprise != False and surprise_hist[-1] != 11:
        my_move = surprise
        click_area(my_move)

    # 其他逻辑
    else:
        # 对方出HighLord就补点Vampire
        if opmove == 16:
            my_move = 10

        # 否则就放Ninja突击
        else:
            '''
            my_moves = supply_mode[opmove-1]
            strategy = np.extract(my_moves <= 14, my_moves)
            for i in strategy:
                if i in (4, 6, 7, 9, 10, 12, 13):
                    my_move = i
                    break
            '''
            my_move = 6

        click_area(my_move)

    # 记录更新历史状态
    turns = turns + 1
    op_hist.append(opmove)
    my_hist.append(my_move)
    surprise_hist.append(surprise)

    # 下一轮到来了再点击
    t_1 = time.time()
    time.sleep(turn_time - (t_1 - t_0) % turn_time)


e:\veighna_studio\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
e:\veighna_studio\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

In [7]:
# 战局记录
import pandas as pd
df = pd.DataFrame({'my_hist': my_hist, 'op_hist': op_hist, 'surprise_hist': surprise_hist})
df.to_csv('record.csv')